# European Call Option BHP

In [3]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
from datetime import datetime

In [ ]:
# Pulling close price as at 16 May 2025 from yfinance
ticker = "BHP.AX"
valuation_date = datetime(2025, 5, 16)
expiry_date = datetime(2027, 9, 15)
target_date_str = valuation_date.strftime("%Y-%m-%d")
data = yf.download(ticker, start=target_date_str, end="2027-09-15", progress=False)
if data.empty:
    raise ValueError(f"No data available for {ticker} on {target_date_str}")
S0 = float(data["Close"].iloc[0])  # Ensure scalar

# Calculate time to maturity dynamically 
T = (expiry_date - valuation_date).days / 365.25  # Year fraction


# Pull volatility (sigma) from cell S15 in sheet "Worksheet"
vol_file = "Rates_Volatilities.xlsx"
from openpyxl import load_workbook

# Open the workbook and access the value from S15
wb = load_workbook(filename=vol_file, data_only=True)
ws = wb["BHP"]
sigma = ws["S15"].value  # This returns the evaluated result (not the formula)

if sigma is None:
    raise ValueError("Volatility (sigma) in cell S15 returned None — please check if Excel formula has been calculated and saved.")

wb.close()
print(f"Spot = {S0:.2f}, Volatility = {sigma:.2%}")

/tmp/ipykernel_14744/2037434503.py:9: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  S0 = float(data["Close"].iloc[0])  # Ensure scalar


Spot = 39.72, Volatility = 20.24%


In [8]:
# Calculate K 
K = 0.98*S0
print(f"Strike price K calculated as 98% of S0: {K}")


#Calculate Risk-free, the maturity of this option was 2.338 years so the 
#3yr risk-free rate was discounted accordingly to align with the maturity of this option
wb = load_workbook(filename=vol_file, data_only=True)
ws = wb["Risk Free"]
r = (ws["F19"].value)/100  # This returns the evaluated result (not the formula)

if r is None:
    raise ValueError("Risk-Free in cell F19 returned None — please check if Excel formula has been calculated and saved.")

wb.close()
print(f"Risk-Free Rate (decimal): {r}")

from instrument_classes import EuropeanCall
EC = EuropeanCall(S0, K, r, T, sigma)
price = EC.price()
print(f"Call Option Price: {price}")

Strike price K calculated as 98% of S0: 38.925601196289065
Risk-Free Rate (decimal): 0.040691012880562066
Call Option Price: 4.32029473813764
